In [23]:
from PIL import Image
import numpy as np

# === INPUT ===
image_path = "DVSKTT_ban_thuc_XIII_19a.jpg"
yolo_box = [0, 0.299350, 0.087522, 0.058178, 0.035042, 0.924133]

# === LOAD IMAGE ===
image = Image.open(image_path)
img_w, img_h = image.size

# === YOLO TO PIXELS ===
_, x_c, y_c, w, h, _ = yolo_box
x_c *= img_w
y_c *= img_h
w *= img_w
h *= img_h

# === SQUARE CROP AROUND CENTER ===
crop_size = max(w, h, 1)  # ensure non-zero
padding = 1.2  # optionally enlarge the box
crop_size *= padding
half_crop = crop_size / 2

left = int(x_c - half_crop)
top = int(y_c - half_crop)
right = int(x_c + half_crop)
bottom = int(y_c + half_crop)

# === CLIP TO IMAGE BOUNDS ===
left = max(0, left)
top = max(0, top)
right = min(img_w, right)
bottom = min(img_h, bottom)

# === CROP AND RESIZE ===
cropped = image.crop((left, top, right, bottom))
resized = cropped.resize((128, 128), Image.LANCZOS)

# === SAVE OR SHOW ===
resized.save("cropped_128x128.jpg")
# resized.show()

In [ ]:
#TORCHVISION TO ENHANCE
from torchvision import transforms
from PIL import Image

# Load your cropped image
img = Image.open("cropped_128x128.jpg").convert("RGB")  # or "L" for grayscale

# Define the transform
transform = transforms.Compose([
    transforms.Resize(size=128),
    transforms.RandomInvert(p=1.0),
])

# Apply it
transformed_img = transform(img)

# Show or save
#transformed_img.show()
transformed_img.save("transformed.jpg")


In [24]:
## OPENCV TO ENHANCE
import cv2

# === Load image in grayscale ===
img = cv2.imread('cropped_128x128.jpg', cv2.IMREAD_GRAYSCALE)

# === Invert if needed (letter is darker than background) ===
# Optional: visualize to decide if this is necessary
# img = cv2.bitwise_not(img)

# === Binarize with adaptive threshold or Otsu's method ===
_, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# === Ensure letter is white and background is black ===
# If the background is white, invert the image
white_ratio = (binary == 255).sum() / binary.size
if white_ratio > 0.5:
    binary = cv2.bitwise_not(binary)

# === Save or visualize ===
cv2.imwrite('binary_128x128.jpg', binary)
# cv2.imshow("Binary", binary); cv2.waitKey(0); cv2.destroyAllWindows()

True